In [72]:
import torchaudio
import librosa
import IPython.display as ipd
import numpy as np
import torch
from mne_bids import BIDSPath, read_raw_bids
import pandas as pd
import numpy as np
import torchaudio
from pathlib import Path
from bm.events import *
from scipy.stats import pearsonr
from scipy.signal import coherence,correlate
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

In [73]:
audio_path ='data/gwilliams2022/download/stimuli/audio/lw1_0.wav'

In [5]:
speech, sr = torchaudio.load(audio_path)
speech = librosa.resample(np.asarray(speech), orig_sr=sr, target_sr=16_000)

In [6]:
model_name = "facebook/wav2vec2-large-xlsr-53"

In [7]:
from transformers import Wav2Vec2Model
from transformers import Wav2Vec2Model, Wav2Vec2Config
from transformers import Wav2Vec2FeatureExtractor

In [8]:
config = Wav2Vec2Config.from_pretrained(model_name)

In [9]:
model =Wav2Vec2Model(config)

In [10]:
feature_extractor =Wav2Vec2FeatureExtractor.from_pretrained(model_name)

In [11]:
out = feature_extractor(speech,return_tensors="pt",
                                     sampling_rate=16_000,
                                     do_normalize=True).input_values

In [12]:
res =model(out, output_hidden_states=True)

In [14]:
out= res.get('hidden_states')

In [63]:
out[0].shape

torch.Size([1, 4793, 1024])

In [23]:
len(out[1])

1

In [27]:
path ='data/gwilliams2022/download/'

In [28]:
bids_path = BIDSPath(
            subject="05",
            session="0",
            task="0",
            root=path,
            datatype="meg",
        )

In [29]:
raw = read_raw_bids(bids_path) 

Extracting SQD Parameters from data/gwilliams2022/download/sub-05/ses-0/meg/sub-05_ses-0_task-0_meg.con...
Creating Raw.info structure...
Setting channel info structure...
Creating Info structure...
Ready.
Reading events from data/gwilliams2022/download/sub-05/ses-0/meg/sub-05_ses-0_task-0_events.tsv.
Reading channel info from data/gwilliams2022/download/sub-05/ses-0/meg/sub-05_ses-0_task-0_channels.tsv.
The stimulus channel "STI 014" is present in the raw data, but not included in channels.tsv. Removing the channel.


/tmp/ipykernel_1839997/3824812548.py:1: RuntimeWarning: The unit for channel(s) MISC 001, MISC 002, MISC 003, MISC 004, MISC 005, MISC 006, MISC 007, MISC 008, MISC 009, MISC 010, MISC 011, MISC 012, MISC 013, MISC 014, MISC 015, MISC 016, MISC 017, MISC 018, MISC 019, MISC 020, MISC 021, MISC 022, MISC 023, MISC 024, MISC 025, MISC 026, MISC 027, MISC 028, MISC 029, MISC 030, MISC 031, MISC 032 has changed from V to NA.
  raw = read_raw_bids(bids_path)


In [30]:
picks = dict(meg=True, eeg=False, stim=False, eog=False, ecg=False, misc=False)
raw = raw.pick_types(**picks)

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


In [40]:
brain_activity =raw.get_data()

In [98]:
raw.resample(sfreq=50)

Measurement date,"January 01, 2000 00:00:00 GMT"
Experimenter,mne_anonymize
Participant,sub-05
Digitized points,Not available
Good channels,208 Magnetometers
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,50.00 Hz
Highpass,0.03 Hz
Lowpass,25.00 Hz


In [100]:
brain_activity2 =raw.get_data()

In [34]:
brain_activity[0,23506:].shape

(360494,)

In [35]:
1000/49

20.408163265306122

In [36]:
360494//20

18024

In [39]:
speech.shape[1] //16

95881

In [70]:
speech49 = librosa.resample(np.asarray(speech), orig_sr=16_000, target_sr=50)

In [103]:
speech1000= librosa.resample(np.asarray(speech), orig_sr=16_000, target_sr=1000)

In [95]:
pearsonr(out[0][0,:,0].detach().numpy(), speech49[0, :out[0].shape[1]])

PearsonRResult(statistic=0.02029936147401987, pvalue=0.15998162222246606)

In [ ]:
raw.resample()

In [105]:
brain_activity.shape

(208, 384000)

In [104]:
speech1000.shape

(1, 95882)

In [101]:
brain_activity2.shape

(208, 19200)